In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import yfinance as yf

In [2]:
# Load the stock price data
data = yf.download(tickers='TSM',start='2003-01-01',end='2023-04-17')
data_df = data['Close']
print(data.shape)

[*********************100%***********************]  1 of 1 completed
(5106, 6)


In [3]:
# Split the data into training and testing sets
train_data = data[:2000]
test_data = data[2000:]

print(train_data.shape,'\n',test_data.shape)

(2000, 6) 
 (3106, 6)


In [4]:
# Define the input and output sequences
def create_sequences(data, sequence_length):
    inputs = []
    outputs = []
    for i in range(len(data) - sequence_length):
        inputs.append(data[i:i+sequence_length])
        outputs.append(data[i+sequence_length])
    return np.array(inputs), np.array(outputs)

sequence_length = 30
x_train, y_train = create_sequences(train_data['Close'], sequence_length)
x_test, y_test = create_sequences(test_data['Close'], sequence_length)
print(x_train.shape, y_train.shape)
print('\n')
print(x_test.shape, y_test.shape)



(1970, 30) (1970,)


(3076, 30) (3076,)


In [5]:
x_test[3075:]

array([[89.02999878, 89.79000092, 89.58000183, 88.84999847, 90.83000183,
        88.87999725, 87.25      , 87.73999786, 89.29000092, 86.69999695,
        89.30000305, 89.47000122, 90.04000092, 92.5       , 92.62000275,
        94.58999634, 92.79000092, 90.66000366, 90.80999756, 92.76000214,
        92.47000122, 93.01999664, 92.83999634, 92.16999817, 90.19999695,
        90.23999786, 89.01999664, 89.23999786, 86.87000275, 87.05000305]])

In [6]:
# Define the BRT architecture
class BRT(tf.keras.Model):
    def __init__(self, num_blocks, block_size, num_heads, d_model, dff, dropout_rate):
        super(BRT, self).__init__()
        self.num_blocks = num_blocks
        self.block_size = block_size
        self.d_model = d_model
        
        # Define the transformer blocks
        self.transformer_blocks = []
        for i in range(num_blocks):
            self.transformer_blocks.append(tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model))
            self.transformer_blocks.append(tf.keras.layers.Dense(units=d_model, activation='relu'))
            self.transformer_blocks.append(tf.keras.layers.Dropout(rate=dropout_rate))
            
        # Define the recurrent layer
        self.recurrent_layer = tf.keras.layers.LSTM(units=d_model)
        
    def call(self, inputs,**kwargs):
        # Split the input sequence into blocks
        inputs = tf.reshape(inputs, [-1, self.block_size, self.d_model])
        
        # Apply the transformer blocks to each block
        for transformer_block in self.transformer_blocks:
            inputs = transformer_block(inputs)
        
        # Apply the recurrent layer to the block outputs
        outputs = self.recurrent_layer(inputs)
        
        return outputs


In [7]:
# Instantiate the BRT
num_blocks = 4
block_size = sequence_length
num_heads = 8
d_model = 128
dff = 512
dropout_rate = 0.1

brt = BRT(num_blocks, block_size, num_heads, d_model, dff, dropout_rate)

# Compile the BRT
brt.compile(optimizer='adam', loss='mse')

# Train the BRT
brt.fit(x_train, y_train, epochs=10, batch_size=32)

# Evaluate the BRT on the test data
test_loss = brt.evaluate(x_test, y_test)

# Make predictions using the BRT
predictions = brt.predict(x_test)

# Plot the predictions and actual values
import matplotlib.pyplot as plt

plt.plot(predictions, label='Predictions')
plt.plot(y_test, label='Actual Values')
plt.legend()
plt.show()


Epoch 1/10


TypeError: in user code:

    File "c:\Users\johnn\anaconda3\envs\tf_2.3_py_3.7\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\johnn\anaconda3\envs\tf_2.3_py_3.7\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\johnn\anaconda3\envs\tf_2.3_py_3.7\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\johnn\anaconda3\envs\tf_2.3_py_3.7\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\johnn\anaconda3\envs\tf_2.3_py_3.7\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\johnn\AppData\Local\Temp\__autograph_generated_filem29z04b4.p

In [ ]:
from pydataset import data
import statsmodels.api as sm
from statsmodels.genmod import families

# Load the geepack dataset
data_geepack = data("geepack")

# Create the GEE model
model = sm.GEE.from_formula("y ~ age + trt + base", groups="subject", data=data_geepack, family=families.Poisson())

# Fit the GEE model
result = model.fit()

# Print the results
print(result.summary())


Exception: Not valid dataset name and no similar found! Try: data() to see available.

In [ ]:
pip install statsmodels

     ---------------------------------------- 9.2/9.2 MB 1.1 MB/s eta 0:00:00
     ------------------------------------- 233.8/233.8 kB 14.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pydataset

     ---------------------------------------- 15.9/15.9 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pydataset: filename=pydataset-0.2.0-py3-none-any.whl size=15939425 sha256=3893e7e57dfeadd374be0709f4873bd3953f7346ec1cd4d66fc516b1d3364a88
  Stored in directory: c:\users\johnn\appdata\local\pip\cache\wheels\8f\0a\01\a4d4b557a0be677dd1d1c5bc5374047254a0f322790e02a3af
Successfully built pydataset
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install geepack

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement geepack (from versions: none)
ERROR: No matching distribution found for geepack
